이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 GloVe 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/22885  

이 자료는 2021년 10월 14일에 마지막으로 테스트되었습니다.

In [ ]:
pip install glove_python_binary

     |████████████████████████████████| 948 kB 5.2 MB/s 


In [ ]:
pip list | grep glove

glove-python-binary           0.2.0


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fb1d8b19cd0>)

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
from glove import Corpus, Glove

corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [ ]:
print(glove.most_similar("man"))

[('woman', 0.9592558146481998), ('guy', 0.8879702965842972), ('girl', 0.8681301312495904), ('kid', 0.840433557687172)]


In [ ]:
print(glove.most_similar("boy"))

[('girl', 0.9431204365432643), ('woman', 0.8604915757383735), ('man', 0.8337024111794784), ('kid', 0.83161801327567)]


In [ ]:
print(glove.most_similar("university"))

[('harvard', 0.8951920214630628), ('mit', 0.853900380131127), ('cambridge', 0.8527690785573532), ('stanford', 0.8430588170027818)]


In [ ]:
print(glove.most_similar("water"))

[('air', 0.8279656964348078), ('clean', 0.8254072089250712), ('fresh', 0.8167668389724283), ('electricity', 0.8164066660033333)]


In [ ]:
print(glove.most_similar("physics"))

[('chemistry', 0.887845484385345), ('economics', 0.8701227264628881), ('mathematics', 0.8566791617676138), ('beauty', 0.8564305737208275)]


In [ ]:
print(glove.most_similar("muscle"))

[('tissue', 0.8544125652057311), ('nerve', 0.8302245727838146), ('skeletal', 0.7670955998540845), ('stem', 0.762369858460031)]


In [ ]:
print(glove.most_similar("clean"))

[('fresh', 0.8308020772677666), ('water', 0.8254072089250712), ('heat', 0.8168824901537102), ('wind', 0.7785974847572267)]
